In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Parameters
thickness_neoprene = 0.003175  # Neoprene thickness (in)
thickness_thinsulate = 1.0  # Thinsulate thickness (in)
thickness_plywood = 0.25  # Plywood thickness (in)
T_ambient = 95  # Ambient temperature (°F) - Standard EER test conditions
#T_ambient = 110  # Ambient temperature (°F) - Extreme conditions
T_bulk_interior = 80  # Bulk air temperature (°F) - Standard EER test conditions
eer = 10.0  # Energy Efficiency Ratio (EER) of the air conditioner (BTU/hr/W)

# Inputs
I = 1000  # Incident radiation (W/m²), set to 0 for no radiation
emissivity = 0.9  # Emissivity
reflectivity = 0.5  # Reflectivity
h_ext = 5  # External convective coefficient (W/m²·K)
h_int = 10  # Internal convective coefficient (W/m²·K)

# Constants
sigma = 5.67e-8  # Stefan-Boltzmann constant (W/m^2·K^4)
A = 1.0  # Surface area (m²)
k_neoprene = 0.036  # Thermal conductivity of neoprene (W/m·K)
k_thinsulate = 0.0387  # Thermal conductivity of thinsulate (W/m·K)
k_plywood = 0.1153  # Thermal conductivity of plywood (W/m·K)

# Function to convert inches to meters
def inches_to_meters(inches):
    return inches * 0.0254

# Function to convert Fahrenheit to Kelvin
def fahrenheit_to_kelvin(fahrenheit):
    return (fahrenheit - 32) * 5/9 + 273.15

# Function to convert Kelvin to Fahrenheit
def kelvin_to_fahrenheit(kelvin):
    return (kelvin - 273.15) * 9/5 + 32

# Function to convert Watts to BTU/hr
def watts_to_btu_per_hr(watts):
    return watts * 3.412

# Function to calculate energy (in watts) for cooling based on EER
def energy_to_cool(eer, Q_removed_btu_per_hr):
    return Q_removed_btu_per_hr / eer

# Conversions
thickness_neoprene_m = inches_to_meters(thickness_neoprene)
thickness_thinsulate_m = inches_to_meters(thickness_thinsulate)
thickness_plywood_m = inches_to_meters(thickness_plywood)
T_ambient_k = fahrenheit_to_kelvin(T_ambient)  # Ambient temperature in Kelvin
T_bulk_interior_k = fahrenheit_to_kelvin(T_bulk_interior)  # Bulk air temperature in Kelvin

# Function to calculate total thermal resistance
def total_resistance(h_conv):
    #R_neoprene1 = thickness_neoprene_m / (k_neoprene * A)
    R_neoprene1 = 0
    #R_thinsulate = thickness_thinsulate_m / (k_thinsulate * A)
    R_thinsulate = 0
    R_plywood = thickness_plywood_m / (k_plywood * A)
    #R_neoprene2 = thickness_neoprene_m / (k_neoprene * A)
    R_neoprene2 = 0
    R_conv = 1 / (h_conv * A)
    #return R_neoprene1 + R_thinsulate + R_plywood + R_neoprene2 + R_conv
    return R_neoprene1 + R_thinsulate/2 + R_plywood + R_neoprene2 + R_conv


# Heat balance function for surface temperature
def heat_balance(T_surface, I, emissivity, reflectivity, h_ext, T_ambient_k):
    Q_absorbed = I * (1 - reflectivity)  
    Q_radiated = emissivity * sigma * A * (T_surface**4 - T_ambient_k**4)
    Q_convected = h_ext * A * (T_surface - T_ambient_k)
    Q_net = Q_absorbed - Q_radiated - Q_convected
    return Q_net

# Calculate equilibrium surface temperature
def equilibrium_temperature(I, emissivity, reflectivity, h_ext, T_ambient_k):
    T_surface_initial = T_ambient_k + 5  # Initial guess slightly above ambient
    T_surface_equilibrium = fsolve(heat_balance, T_surface_initial, args=(I, emissivity, reflectivity, h_ext, T_ambient_k))
    return T_surface_equilibrium[0]

# Calculate energy to remove to maintain internal air temperature
def energy_to_remove(I, emissivity, reflectivity, h_ext, h_int, T_ambient_k, T_bulk_interior_k):
    T_surface_k = equilibrium_temperature(I, emissivity, reflectivity, h_ext, T_ambient_k)
    R_total = total_resistance(h_int)
    Q_removed = (T_surface_k - T_bulk_interior_k) / R_total
    return T_surface_k, Q_removed

# Rough Estimate of delta (shaded vs unshaded) - inaccurate
def rough_factor(T_surface_k, T_surface_k_shade):
    return (T_surface_k / T_surface_k_shade)

# Run the model
T_surface_result_k, Q_removed_result = energy_to_remove(I, emissivity, reflectivity, h_ext, h_int, T_ambient_k, T_bulk_interior_k)

# Run with no radiation
T_surface_result_k_no_rad, Q_removed_result_no_rad = energy_to_remove(0, emissivity, reflectivity, h_ext, h_int, T_ambient_k, T_bulk_interior_k)

# Run with 90% shading
T_surface_result_k_shade, Q_removed_result_shade = energy_to_remove(0.1 * I, emissivity, reflectivity, h_ext, h_int, T_ambient_k, T_bulk_interior_k)

# Convert results to Fahrenheit
T_surface_result = kelvin_to_fahrenheit(T_surface_result_k)
T_surface_result_no_rad = kelvin_to_fahrenheit(T_surface_result_k_no_rad)
T_surface_result_shade = kelvin_to_fahrenheit(T_surface_result_k_shade)

# Calculate energy to cool
Q_removed_btu_per_hr = watts_to_btu_per_hr(Q_removed_result)
energy_cooling = energy_to_cool(eer, Q_removed_btu_per_hr)
Q_removed_btu_per_hr_no_rad = watts_to_btu_per_hr(Q_removed_result_no_rad)
energy_cooling_no_rad = energy_to_cool(eer, Q_removed_btu_per_hr_no_rad)
Q_removed_btu_per_hr_shade = watts_to_btu_per_hr(Q_removed_result_shade)
energy_cooling_shade = energy_to_cool(eer, Q_removed_btu_per_hr_shade)

# Output results
print(f"Equilibrium Surface Temperature: {T_surface_result:.2f} °F")
print(f"Energy to Remove: {Q_removed_result:.2f} W")
print(f"Energy to Cool: {energy_cooling:.2f} W")
print(f"Equilibrium Surface Temperature (no radiation): {T_surface_result_no_rad:.2f} °F")
print(f"Energy to Remove (no radiation): {Q_removed_result_no_rad:.2f} W")
print(f"Energy to Cool (no radiation): {energy_cooling_no_rad:.2f} W")
print(f"Equilibrium Surface Temperature (90% shading): {T_surface_result_shade:.2f} °F")
print(f"Energy to Remove (90% shading): {Q_removed_result_shade:.2f} W")
print(f"Energy to Cool (90% shading): {energy_cooling_shade:.2f} W")
print(f"Rough Factor (shaded vs unshaded): {rough_factor(T_surface_result_k, T_surface_result_k_shade):.2f}")
print(f"Factor (shaded vs unshaded) - Energy: {Q_removed_result/Q_removed_result_shade:.2f}")


# Top Level Header

Lorum ipsum dolor sit amet, consectetur adipiscing elit. Nullam nec pur... 

## Second Level Header

Lorum ipsum dolor sit amet, consectetur adipiscing elit. Nullam nec pur...

Per reference @IEC_TR_62240-1, blah blah blah.

# References